In [0]:
dbutils.widgets.text('fromdate', '')
dbutils.widgets.text('enddate', '')

fromdate = dbutils.widgets.get('fromdate')
enddate = dbutils.widgets.get('enddate')


from time import sleep
import requests
import re
import json
import datetime
import gzip
from requests_aws4auth import AWS4Auth


# ebazaaris
C_KEY = 'amzn1.application-oa2-client.10a.....'
C_SECRET_KEY = 'ab61c932490a........34efc34ad8'
C_REFRESH_TOKEN = 'Atzr|IwEBIIHldgx2EZ_PfbXqn........TIwkqJKb8'


markets = [
    ('A1PA6795UKMFR9', 'de'),
    ('A13V1IB3VIYZZH', 'fr'),
    ('A1805IZSGTT6HS', 'nl'),
    ('A1C3SOZRARQ6R3', 'pl'),
    ('A1F83G8C2ARO7P', 'co.uk'),
    ('A1RKKUPIHCS9HS', 'es'),
    ('A2NODRKZP88ZB9', 'se'),
    ('APJ6JRA9NG5V4', 'it')]

In [0]:
def get_datetime():
    return datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")



def get_access_token():
    global C_KEY, C_SECRET_KEY, C_REFRESH_TOKEN
    r = requests.post(f"https://api.amazon.com/auth/o2/token", data={
        "grant_type":"refresh_token",
        "refresh_token":C_REFRESH_TOKEN,
        "client_id": C_KEY,
        "client_secret": C_SECRET_KEY
    })

    # print(r.json())

    return r.json()['access_token']




def assume_role():
    r = requests.get(f"https://sts.amazonaws.com?Version=2011-06-15&Action=AssumeRole&RoleSessionName=Test&RoleArn=arn:aws:iam::466....17:role/SellingPartnerApiRole&DurationSeconds=3600",
        auth=AWS4Auth('AKIAW.....RR6N', 'g6VDrBF.....1eV', 'us-east-1', 'sts'))

    return (re.findall('<AccessKeyId>(.+)</AccessKeyId>', r.text)[0], re.findall('<SecretAccessKey>(.+)</SecretAccessKey>', r.text)[0], re.findall('<SessionToken>(.+)</SessionToken>', r.text)[0])




def g(rel_url):
    global access_token, key, secret_key, session_token

    r = requests.get(f"https://sellingpartnerapi-eu.amazon.com{rel_url}",
        headers={'x-amz-access-token': access_token},
        auth=AWS4Auth(key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token))

    return(r.json())




def gg(rel_url, params):
    global access_token, key, secret_key, session_token

    r = requests.get(f"https://sellingpartnerapi-eu.amazon.com{rel_url}",
        params=params,
        headers={'x-amz-access-token': access_token},
        auth=AWS4Auth(key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token))

    print(r.text)

    return(r.json())




def gd(url):
    r = requests.get(url)
    return(r.content)




def p(rel_url, data):
    global access_token, key, secret_key, session_token

    r = requests.post(f"https://sellingpartnerapi-eu.amazon.com{rel_url}",
        data=json.dumps(data),
        headers={
            'x-amz-access-token': access_token,
            'Content-Type': 'application/json'
        },
        auth=AWS4Auth(key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token))

    return (r.json())


access_token = get_access_token()
key, secret_key, session_token = assume_role()




def genn(REPORT_TYPE, market, start, end):
    c_r = p('/reports/2021-06-30/reports', {
        "marketplaceIds": [market],
        'reportType': REPORT_TYPE,
        'dataStartTime': start,
        'dataEndTime': end,
        "reportOptions": {
            "dateGranularity": "DAY",
            "asinGranularity": "SKU",
            "reportPeriod": "DAY"
        }
    })

    # print(c_r)
    return c_r['reportId']



def down(report_id):
    g_r = g(f'/reports/2021-06-30/reports/{report_id}')
    doc_id = g_r['reportDocumentId']
    d_r = g(f'/reports/2021-06-30/documents/{doc_id}')
    bt = (gd(d_r['url']))
    file = (gzip.decompress(bt).decode('utf-8'))
    return file



def get_report(title, market, fileName, dt):
    report_id = genn(title, market, dt, dt)
    sleep(100)
    file = down(report_id)
    dbutils.fs.put('dbfs:/mnt/' + containerName + '/' + f'{fileName}.json', file , overwrite = True )
    return file


In [0]:
containerName = "amazon-seller-api"
storageAccountName = "blobsaless"
sas = "sp=racwdli&st=2022-10-10T14:15:25Z&se=2030-10-1.........X6E%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/amazon-seller-api")
except:
    pass
    
try:
    dbutils.fs.mount( source = "wasbs://{}@{}.blob.core.windows.net".format(containerName,storageAccountName),
                      mount_point = "/mnt/amazon-seller-api",
                      extra_configs = {config : sas}
                    )
except:
    pass

/mnt/amazon-seller-api has been unmounted.

In [ ]:
# Generate and Download Report for each marketplace in the list and for each date between range

cur = datetime.date.fromisoformat(fromdate)
end = datetime.date.fromisoformat(enddate)
i = 0


while cur < end:
    print('Getting: ', str(cur))

    if i % 2 == 0:
        access_token = get_access_token()

    for m in markets[:]:
        get_report('GET_SALES_AND_TRAFFIC_REPORT', m[0], f'reports/GET_SALES_AND_TRAFFIC_REPORT__{str(cur)}__{m[1]}', str(cur))
    
    cur = cur + datetime.timedelta(days=1)
    i += 1

    
print('Done!!!')